#### Question 1

In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
np.random.seed(100)

### Change T and k values and run 1000 simulations under each group        
T = [10,25,50,75,100]
sig=0.5
bigT=T*2
mu_x=1
sig_x=2
k=4
b0=0.5
b1=1
nsim = 1000
kvec = [2,4,8,10]
Tvec = [25,50,75,100]
outp =np.zeros((nsim,len(Tvec),len(kvec),5))
for tdx,T in enumerate(Tvec):
    bigT = T*2
    for kdx,k in enumerate(kvec):
        for isim in range(nsim):
            e = np.random.normal(0,sig,size=(bigT,1))
            x = np.random.normal(mu_x,sig_x,size=(bigT,1))
            x_t=x
            y=b0+(b1*x_t)+e
            x_t = sm.add_constant(x)
            sample = np.random.permutation(bigT)
            train = sample[:int(bigT/2)]
            test = sample[int(bigT/2):]
            ytr = y[train]
            xtr = x_t[train]
            model=smf.OLS(y[train],x_t[train]).fit()
            z=model.predict(x_t[test])
            mspe_true = ((y[test]-model.predict(x_t[test]))**2).mean()
            sample = np.random.permutation(T)
            mspe =np.zeros(k)
            for j in range(k):    
                train_l = sample[:int((k-1-j)*T/k)]
                train_u = sample[int((k-j)*T/k):]
                train = np.concatenate((train_l,train_u),0)
                test = sample[int((k-1-j)*T/k):int((k-j)*T/k)]
                model = smf.OLS(ytr[train],xtr[train]).fit()
                mspe[j] = ((ytr[test]-model.predict(xtr[test]))**2).mean()
            mspe_cv = mspe.mean()
            outp[isim,tdx,kdx] = np.array((T,k,mspe_true,mspe_cv,mspe_cv-mspe_true))

## Convert Array to DataFrame for analysis
df=pd.DataFrame(np.vstack(outp[:,:,:].mean(0)))
df.columns=['T','k','mspe_actual','mspe_crossvalid','difference']
df

## Comments:
# MSPE Actual is around 8 for all T values  - 10,25,50,75,100
# MSPE Cross Validation decreases for evey T and k combination as 
# number of splits increases ---> as k increases more data are feeded
# into training the model and MSPE cross validation decreases
# From the simulation results we can comment that for any value of T,
# k=2 splits seems optimum as difference between MSPE Actual 
# MSPE Cross Validation seems minimum


,T,k,mspe_actual,mspe_crossvalid,difference
0,25.0,2.0,7.978254,7.565869,-0.412385
1,25.0,4.0,7.872653,6.951630,-0.921023
2,25.0,8.0,7.910748,5.649310,-2.261438
3,25.0,10.0,8.071310,4.926897,-3.144414
4,50.0,2.0,8.104254,7.979666,-0.124588
5,50.0,4.0,8.024761,7.601483,-0.423278
6,50.0,8.0,8.061416,6.973445,-1.087971
7,50.0,10.0,8.156120,6.629674,-1.526446
8,75.0,2.0,8.079486,8.002217,-0.077269
9,75.0,4.0,8.162480,7.786868,-0.375612


#### Question 2

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
np.random.seed(100) 
pd.set_option('display.max_columns',10)
pd.set_option('display.width',None)
## Betas, Bootstraps are variable
bvec=[-0.1,0,0.1,0.2]
T=100
nsim = 1000
b0 = 1
boots=[10,20,30,50]
sig=0.5
mu_x=1
outp =np.zeros((len(bvec),len(boots),6))
outp.shape
for bdx,b1 in enumerate(bvec):      
    for btx,boot in enumerate(boots):
        reject_boot = 0
        reject_both = 0
        reject_orig=0
        for isim in range(nsim):
            e = np.random.normal(0,sig,size=(T,1))
            x = np.random.normal(mu_x,sig,size=(T,1))
            y=b0+(b1*x)+e
            x = sm.add_constant(x)
            model_orig=smf.OLS(y,x).fit()
            # Create confindence interval for b1
            ci=np.zeros(int(boot))
            for b in range(boot):  ## Bootstrapping to find CI
                # Create a resample index
                resamp=np.random.randint(T, size=int(0.5*T)) #Re-sampling with replacement
                model = smf.OLS(y[resamp],x[resamp]).fit()
                ci[b] = model.params[1]
                #print(model.params[1])
            ci.sort()
            ci_lower = ci[int(boot*0.025)]
            ci_upper = ci[int(boot*0.975)]
            ## Empirical t-tests - Reject Null when CI follows a trend
            if not(0>=ci_lower and 0<=ci_upper): reject_boot+=1
            if not(0>=ci_lower and 0<=ci_upper) and model_orig.pvalues[1]<0.05: reject_both+=1
            if model_orig.pvalues[1]<0.05: reject_orig+=1
        outp[bdx,btx] = np.array((b1,boot,nsim,reject_orig,reject_boot,reject_both))
## Convert to Dataframe for analysis
df=pd.DataFrame(np.vstack(outp))
df.columns=['BETA','BOOTSTRAP','No_of_Simulation','Actual_Rej','Bootstrap_Rej','Both_rej']
df['Power of the Test(when Beta=0, its Size of Test)']=(df['Both_rej']/df['No_of_Simulation'])*100
df

#### Question 3

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

In [11]:
btc=pd.read_csv("bitcoin.csv")
snp500=pd.read_csv("SP500.csv")

gold=pd.read_csv("GOLDAMGBD228NLBM.csv")

dexus=pd.read_csv("DEXUSEU.csv")

dcoil=pd.read_csv("DCOILWTICO.csv")

FileNotFoundError: File b'bitcoin.csv' does not exist

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from IPython.display import display
display(btc.head())
display(snp500.head())
display(gold.head())
display(dexus.head())
display(dcoil.head())

In [ ]:
from IPython.display import display
display(btc.shape)
display(snp500.shape)
display(gold.shape)
display(dexus.shape)
display(dcoil.shape)

In [ ]:
btc['DATE'] = pd.to_datetime(btc.DATE)
snp500['DATE'] = pd.to_datetime(snp500.DATE)
gold['DATE'] = pd.to_datetime(gold.DATE)
dexus['DATE'] = pd.to_datetime(dexus.DATE)
dcoil['DATE'] = pd.to_datetime(dcoil.DATE)


In [ ]:
data=pd.merge(pd.merge(pd.merge(pd.merge(btc,dexus,on='DATE'),dcoil,on='DATE'),snp500,on='DATE'),gold,on='DATE')

In [ ]:
data.head()

In [ ]:
import numpy as np
data=data.replace('.', np.NaN)
data = data.fillna(method='pad')
data=data[1:]


In [ ]:
print(data.info())

In [ ]:
data['DEXUSEU'] = data.DEXUSEU.astype(float)
data['DCOILWTICO'] = data.DCOILWTICO.astype(float)
data['SP500'] = data.SP500.astype(float)
data['GOLDAMGBD228NLBM'] = data.GOLDAMGBD228NLBM.astype(float)

In [ ]:
columns=['Close Price', 'DEXUSEU', 'DCOILWTICO', 'SP500', 'GOLDAMGBD228NLBM']
columns

In [ ]:
data[[x for x in columns] + 
   ['DATE']].groupby('DATE').sum().plot()

In [ ]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
series = Series.from_csv('bitcoin.csv', header=0)
plot_acf(series)
pyplot.show()

In [ ]:
import datetime

date_before = datetime.date(2016, 12, 31)
bd = data[data['DATE'] > pd.to_datetime(date_before)]

In [ ]:
bd.head()

In [ ]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
series1 = bd.iloc[:,1]
plot_acf(series, lags=50)
pyplot.show()

In [ ]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
series1 = bd.iloc[:,1]
plot_pacf(series, lags=50)
pyplot.show()

In [ ]:
from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(series1)
pyplot.show()

In [ ]:
series2 = data.iloc[:,2]
from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(series2)
pyplot.show()

In [ ]:
from numpy import log
series1=log(series1)

In [ ]:
# Plot histogram
series2.hist()
plt.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.stattools

for i in range(0,30):
    print(statsmodels.tsa.stattools.kpss(series, regression='ct', lags=i, store=False))

In [ ]:
statsmodels.tsa.stattools.kpss(series, regression='ct', lags=None, store=False)

In [ ]:
import pandas
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import math
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
lnprice=np.log(data['Close Price'])
lnprice
plt.plot(lnprice)
plt.show()
acf_1 =  acf(lnprice)[1:20]
plt.plot(acf_1)
plt.show()
test_df = pandas.DataFrame([acf_1]).T
test_df.columns = ['Pandas Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1 =  pacf(lnprice)[1:20]
plt.plot(pacf_1)
plt.show()
test_df = pandas.DataFrame([pacf_1]).T
test_df.columns = ['Pandas Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
result = ts.adfuller(lnprice, 1)
result

In [ ]:
lnprice_diff=lnprice-lnprice.shift()
diff=lnprice_diff.dropna()
acf_1_diff =  acf(diff)[1:20]
test_df = pandas.DataFrame([acf_1_diff]).T
test_df.columns = ['First Difference Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1_diff =  pacf(diff)[1:20]
plt.plot(pacf_1_diff)
plt.show()

In [ ]:
price_matrix=lnprice.as_matrix()
model = ARIMA(price_matrix, order=(3,1,0))
model_fit = model.fit(disp=0)


In [ ]:
print(model_fit.summary())

In [ ]:
# plot residual errors
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 

X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(3,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
X = data.values

In [ ]:
series

In [ ]:
# create a differenced series
import numpy
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return numpy.array(diff)
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# print summary of fit model
print(model_fit.summary())

In [ ]:
# create a differenced series
import numpy
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return numpy.array(diff)
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(5,2,2))
model_fit = model.fit(disp=0)
# print summary of fit model
print(model_fit.summary())

In [ ]:
forecast = model_fit.predict()

In [ ]:
forecast

In [ ]:
predictions=model_fit.predict(1245, 1275, typ='levels')
predictions

In [ ]:
data.tail()

In [ ]:
price_matrix

In [ ]:
lnprice

In [ ]:
import scipy
scipy.signal.periodogram(series, fs=1.0, window=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)

In [ ]:
newbitcoin = data[488:843]
newbitcoin

In [ ]:
from pandas import Series
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot

price = newbitcoin.iloc[:,-1].values
date = newbitcoin.iloc[:,0].values
plot_acf(price, ax=pyplot.gca())
pyplot.show()

In [ ]:
plot_pacf(price, ax=pyplot.gca())
pyplot.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
xarima = ARIMA(price,order=(1,0,0)).fit()

In [ ]:
#from statsmodels.tsa.arima_model import ARMAResults

x= xarima.forecast(steps=30)[0]
x

